### LangChain 을 사용해서, llm 을 사용하는 순서.

1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰 수 초과로 답변을 생성하지 못할 수 있고,
    - 문서가 길면 (input 이 길면) 답변 생성이 오래걸림.
3. 임베딩 -> 벡터 데이터베이스로 저장.
4. 질문을 하면, 벡터 데이터베이스에서 유사도 검색.
5. 유사도 검색에서 가져온 문서를 llm 모델에 질문과 같이 전달.

## 필요한 패키지 설치.

%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma

## Knowledge Base 구성을 위한 데이터 생성.

- RecursiveCharacterTextSplitter를 활용한 데이터 chunking
    - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
    - 비용 감소와 답변 생성시간 감소의 효과
    - LangChain에서 다양한 TextSplitter들을 제공
- chunk_size 는 split 된 chunk의 최대 크기
- chunk_overlap은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

### Open AI Embedding 을 사용해서, 임베딩 (Vector 화)

In [4]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
# model 은 large 모델을 사용하는게 좋음 (한글이 인식되는 정도가 다르기 때문)
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

### 데이터베이스 입력

%pip install -qU langchain-pinecone pinecone-notebooks

In [6]:
import getpass
import os

from pinecone import Pinecone

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

/Users/kcyang/Documents/ai_models/inf-llm/.venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [10]:
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [22]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)


[Document(id='084422cd-6700-405a-9bf4-1dbc9e023585', metadata={'source': './tax.docx'}, page_content='1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는 근로의 제공으로 인한 부상ㆍ질병ㆍ사망과 관련하여 근로자나 그 유족이 받는 배상ㆍ보상 또는 위자(慰藉)의 성질이 있는 급여\n\n라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금

In [23]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [24]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/kcyang/Documents/ai_models/inf-llm/.venv/lib/python3.12/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [25]:
from langchain.chains import RetrievalQA


qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [26]:
# LangChain 권장사항에 따라 강의 코드와 다르게 `.invoke()`를 사용합니다
ai_message = qa_chain.invoke({"query": query})

In [27]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '죄송하지만 연봉 5천만 원인 직장인의 소득세를 구체적으로 계산할 수 있는 기준이나 정보가 제공되지 않았습니다. 소득세는 근로소득 공제, 인적 공제, 특별 세액공제 등 다양한 변수를 고려해 계산됩니다. 정확한 정보를 얻으려면 국세청의 소득세 계산기를 참조하거나 전문 세무사와 상담하는 것이 좋습니다.'}